In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from surprise import Dataset, print_perf, evaluate, SVD, GridSearch, BaselineOnly, SVDpp

In [3]:
data = Dataset.load_builtin('ml-100k')
data.split()

In [5]:
algo = SVD(lr_all = .005,n_epochs = 100, reg_all = .09)

In [ ]:
evaluate(algo,data)

Evaluating RMSE, MAE of algorithm SVDpp.

------------
Fold 1
RMSE: 0.9516
MAE:  0.7529
------------

In [70]:
evaluate(algo,data)

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.9759
MAE:  0.7689
------------
Fold 2
RMSE: 0.9805
MAE:  0.7746
------------
Fold 3
RMSE: 0.9772
MAE:  0.7698
------------
------------
Mean RMSE: 0.9778
Mean MAE : 0.7711
------------
------------


CaseInsensitiveDefaultDict(list,
                           {u'mae': [0.76887090436082539,
                             0.77464512982494571,
                             0.76981157520526899],
                            u'rmse': [0.97587061123025087,
                             0.98047418143896659,
                             0.97720259442259894]})

# Baseline Model

In [6]:
cd ~/Desktop/Galvanize/dsi/case_study/alt-recommender-case-study/data/movies

/Users/mrd/Desktop/Galvanize/dsi/case_study/alt-recommender-case-study/data/movies


In [7]:
ls

README.txt   links.csv    movies.csv   ratings.csv  tags.csv


In [16]:
df_r = pd.read_csv('ratings.csv')
df_m = pd.read_csv('movies.csv')

In [17]:
df_m['title'][[0,1]]

0    Toy Story (1995)
1      Jumanji (1995)
Name: title, dtype: object

In [18]:
df_r.head(), df_r.shape

(   userId  movieId  rating   timestamp
 0       1       31     2.5  1260759144
 1       1     1029     3.0  1260759179
 2       1     1061     3.0  1260759182
 3       1     1129     2.0  1260759185
 4       1     1172     4.0  1260759205, (100004, 4))

In [19]:
df_rtest = df_r.copy()
df_rtrain = df_r.copy()

In [20]:
df_rtest.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [21]:
df_rtrain['rating'].isnull().value_counts()

False    100004
Name: rating, dtype: int64

In [22]:
df_rtrain['rating'][np.random.randint(0,len(df_r),size = 40000)] = np.nan

/Users/mrd/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
df_rtrain['rating'].isnull().value_counts()

False    67105
True     32899
Name: rating, dtype: int64

In [24]:
df_rtrain.head()

,userId,movieId,rating,timestamp
0,1,31,NaN,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,NaN,1260759185
4,1,1172,4.0,1260759205


In [25]:
df_rtest['rating'].isnull().value_counts()

False    100004
Name: rating, dtype: int64

In [26]:
R_dftest = df_rtest.pivot(index = 'userId', columns ='movieId', values = 'rating')
R_dftrain = df_rtrain.pivot(index = 'userId', columns ='movieId', values = 'rating')

In [27]:
R_dftest.shape, R_dftrain.shape

((671, 9066), (671, 9066))

In [28]:
Rtest = R_dftest.values
Rtrain = R_dftrain.values

In [29]:
import sys
from collections import OrderedDict
import numpy as np

# Define our evaluation function.
def rmse(test_data, predicted):
    """Calculate root mean squared error.
    Ignoring missing values in the test data.
    """
    I = ~np.isnan(test_data)   # indicator for missing values
    N = I.sum()                # number of non-missing values
    sqerror = abs(test_data - predicted) ** 2  # squared error array
    mse = sqerror[I].sum() / N                 # mean squared error
    return np.sqrt(mse)                        # RMSE


# Create a base class with scaffolding for our 3 baselines.
def split_title(title):
    """Change "BaselineMethod" to "Baseline Method"."""
    words = []
    tmp = [title[0]]
    for c in title[1:]:
        if c.isupper():
            words.append(''.join(tmp))
            tmp = [c]
        else:
            tmp.append(c)
    words.append(''.join(tmp))
    return ' '.join(words)


class Baseline(object):
    """Calculate baseline predictions."""

    def __init__(self, train_data):
        """Simple heuristic-based transductive learning to fill in missing
        values in data matrix."""
        self.predict(train_data.copy())

    def predict(self, train_data):
        raise NotImplementedError(
            'baseline prediction not implemented for base class')

    def rmse(self, test_data):
        """Calculate root mean squared error for predictions on test data."""
        return rmse(test_data, self.predicted)
    
    def __str__(self):
        return split_title(self.__class__.__name__)
        
# Implement the 3 baselines.
class UniformRandomBaseline(Baseline):
    """Fill missing values with uniform random values."""

    def predict(self, train_data):
        nan_mask = np.isnan(train_data)
        masked_train = np.ma.masked_array(train_data, nan_mask)
        pmin, pmax = masked_train.min(), masked_train.max()
        N = nan_mask.sum()
        train_data[nan_mask] = np.random.uniform(pmin, pmax, N)
        self.predicted = train_data


class GlobalMeanBaseline(Baseline):
    """Fill in missing values using the global mean."""

    def predict(self, train_data):
        nan_mask = np.isnan(train_data)
        train_data[nan_mask] = train_data[~nan_mask].mean()
        self.predicted = train_data


class MeanOfMeansBaseline(Baseline):
    """Fill in missing values using mean of user/item/global means."""

    def predict(self, train_data):
        nan_mask = np.isnan(train_data)
        masked_train = np.ma.masked_array(train_data, nan_mask)
        global_mean = masked_train.mean()
        user_means = masked_train.mean(axis=1)
        item_means = masked_train.mean(axis=0)
        self.predicted = train_data.copy()
        n, m = train_data.shape
        for i in range(n):
            for j in range(m):
                if np.ma.isMA(item_means[j]):
                    self.predicted[i,j] = np.mean(
                        (global_mean, user_means[i]))
                else:
                    self.predicted[i,j] = np.mean(
                        (global_mean, user_means[i], item_means[j]))

In [30]:
baseline_methods = OrderedDict()
baseline_methods['ur'] = UniformRandomBaseline
baseline_methods['gm'] = GlobalMeanBaseline
baseline_methods['mom'] = MeanOfMeansBaseline

## create a train-test split
test = Rtest
train = Rtrain

train = train.reshape(671,9066)
test = test.reshape(671,9066)

baselines = {}
for name in baseline_methods:
    Method = baseline_methods[name]
    method = Method(train)
    baselines[name] = method.rmse(test)
    print('%s RMSE:\t%.5f' % (method, baselines[name]))

Uniform Random Baseline RMSE:	1.06108
Global Mean Baseline RMSE:	0.60303
Mean Of Means Baseline RMSE:	0.92368
